# Calculation of Envelope Part

## 1. Basic Class

### 1) Side Class

In [1]:
class Side():
    def __init__(self, mns, pls):
        self.mns = mns
        self.pls = pls

## 2. Layer Class

### 1) Solid Layer Class

#### Outline

This class has the properties below:  
- thermal conductivity, W m<sup>-1</sup> K<sup>-1</sup>  
- volumetric specific heat, J m<sup>-3</sup> K<sup>-1</sup>  
- thickness, m  
- division number.

#### Class

In [2]:
class Solid_Layer():
    
    def __init__(self, conductivity, specific_heat, thickness, division_number):
        if conductivity <= 0.0:
            raise ValueError('Conductivity shall be more than 0.0.')
        if specific_heat <= 0.0:
            raise ValueError('Specific Heat shall be more than 0.0.')
        if thickness <= 0.0:
            raise ValueError('Thickness shall be more than 0.0.')
        if division_number <= 0.0:
            raise ValueError('Division number shall be more than 0.')
        self.__conductivity    = conductivity
        self.__specific_heat   = specific_heat
        self.__thickness       = thickness
        self.__division_number = division_number
        self.__delta_x         = thickness / division_number
    
    @property
    def conductivity(self):
        return self.__conductivity
    
    @property
    def specific_heat(self):
        return self.__specific_heat
    
    @property
    def thickness(self):
        return self.__thickness
    
    @property
    def division_number( self ):
        return self.__division_number
    
    @property
    def delta_x(self):
        return self.__delta_x

#### Example

In [3]:
s = Solid_Layer(conductivity=1.2, specific_heat=100.0, thickness=0.12, division_number=10)
s.conductivity, s.specific_heat, s.thickness, s.division_number, s.delta_x

(1.2, 100.0, 0.12, 10, 0.012)

### 2) Air Layer Class

#### Outline

This class has the properties below:  
- thermal resistance, m <sup>2</sup> K W <sup>-1</sup>  

#### Class

In [4]:
class Air_Layer():
    
    def __init__(self, resistance):
        if resistance <= 0.0:
            raise ValueError('Air resistance shall be more than 0.0.')
        self.__resistance = resistance
        
    @property
    def resistance(self):
        return self.__resistance

#### Example

In [5]:
a = Air_Layer(resistance=0.1)
a.resistance

0.1

## 3. Cell Class

### 1) Definition

#### Outline

This class has the properties below:  
- thermal resistance, m <sup>2</sup> K W <sup>-1</sup>  
- capacity, J m <sup>-2</sup> K <sup>-1</sup>
- temperature, degree C

#### Class

In [6]:
class Cell():
    
    def __init__(self, resistance, capacity, temperature):
        self.resistance  = resistance
        self.capacity    = capacity
        self.temp = temperature

### 2) Create Cell Classes from Layer Class

#### Function

In [7]:
def make_cells(layers, initial_temp):

    if type(layers[0]) == Air_Layer:
        raise Exception('First layer should not be air layer.')
    
    if type(layers[len(layers)-1]) == Air_Layer:
        raise Exception('Last layer should not be air layer.')
    
    for i, l in enumerate(layers):
        if type(l) == Air_Layer:
            if type(layers[i+1]) == Air_Layer:
                raise Exception('Air layer should not set next to the air layer.')
    
    cells = []

    for i, layer in enumerate(layers):
        
        if type(layer) == Solid_Layer:
            
            if i == 0:
                cells.append(Cell(resistance  = Side(mns = 'surfaceMns', pls = layer.delta_x / layer.conductivity),
                                  capacity    = layer.specific_heat * layer.delta_x / 2,
                                  temperature = initial_temp
                                 ))
            elif type(layers[i-1]) == Air_Layer:
                cells.append(Cell(resistance  = Side(mns = layers[i-1].resistance, pls = layer.delta_x / layer.conductivity),
                                  capacity    = layer.specific_heat * layer.delta_x / 2,
                                  temperature = initial_temp
                                 ))
            else:
                pass
            
            for j in range(l.division_number - 1):
                cells.append(Cell(resistance  = Side(mns = layer.delta_x / layer.conductivity, pls = layer.delta_x / layer.conductivity),
                                  capacity    = layer.specific_heat * layer.delta_x,
                                  temperature = initial_temp
                                 ))
            
            if i == len(layers) - 1:
                cells.append(Cell(resistance  = Side(mns = layer.delta_x / layer.conductivity, pls = 'surfacePls'),
                                  capacity    = layer.specific_heat * layer.delta_x / 2,
                                  temperature = initial_temp
                                 ))
            elif type(layers[i+1]) == Air_Layer:
                cells.append(Cell(resistance  = Side(mns = layer.delta_x / layer.conductivity, pls = layers[i+1].resistance),
                                  capacity    = layer.specific_heat * layer.delta_x / 2,
                                  temperature = initial_temp
                                 ))
            else:
                cells.append(Cell(resistance  = Side(mns = layer.delta_x / layer.conductivity, pls = layers[i+1].delta_x / layers[i+1].conductivity),
                                  capacity    = layer.specific_heat * layer.delta_x / 2 + layers[i+1].specific_heat * layers[i+1].delta_x / 2,
                                  temperature = initial_temp
                                 ))
                
    return cells

#### Example

In [8]:
cells_exmpl = make_cells(
    [Solid_Layer(conductivity=1.2, specific_heat=100.0, thickness=0.12, division_number=3),
     Air_Layer(resistance=120.0),
     Solid_Layer(conductivity=1.2, specific_heat=100.0, thickness=0.12, division_number=3),
     Solid_Layer(conductivity=1.2, specific_heat=100.0, thickness=0.12, division_number=3),
     Air_Layer(resistance=120.0),
     Solid_Layer(conductivity=1.2, specific_heat=100.0, thickness=0.12, division_number=6),
    ],
    15.0
)

for c in cells_exmpl:
    print( 'R: ' + str(c.resistance.mns) + '/' + str(c.resistance.pls) + ', capacity: ' + str(c.capacity) + ', temperature: ' + str(c.temp) )

R: surfaceMns/0.03333333333333333, capacity: 2.0, temperature: 15.0
R: 0.03333333333333333/0.03333333333333333, capacity: 4.0, temperature: 15.0
R: 0.03333333333333333/0.03333333333333333, capacity: 4.0, temperature: 15.0
R: 0.03333333333333333/0.03333333333333333, capacity: 4.0, temperature: 15.0
R: 0.03333333333333333/0.03333333333333333, capacity: 4.0, temperature: 15.0
R: 0.03333333333333333/0.03333333333333333, capacity: 4.0, temperature: 15.0
R: 0.03333333333333333/120.0, capacity: 2.0, temperature: 15.0
R: 120.0/0.03333333333333333, capacity: 2.0, temperature: 15.0
R: 0.03333333333333333/0.03333333333333333, capacity: 4.0, temperature: 15.0
R: 0.03333333333333333/0.03333333333333333, capacity: 4.0, temperature: 15.0
R: 0.03333333333333333/0.03333333333333333, capacity: 4.0, temperature: 15.0
R: 0.03333333333333333/0.03333333333333333, capacity: 4.0, temperature: 15.0
R: 0.03333333333333333/0.03333333333333333, capacity: 4.0, temperature: 15.0
R: 0.03333333333333333/0.03333333333

## 4. Envelope Class

### 1) Definition

#### Outline

Envelope Class は、以下の情報を持つ。  
- Cell
- Envelope の名前

#### Class

In [9]:
class Envelope():
    
    def __init__(self, nameset, cells, lwaveemit):
        
        def check_longwave_emittance_range(value):
            if value < 0.0 or value > 1.0:
                raise ValueError('Long wave emittance should be in the range between 0.0 and 1.0.')

        check_longwave_emittance_range(lwaveemit.mns)
        check_longwave_emittance_range(lwaveemit.pls)

        self.nameset = nameset
        self.cells = cells
        self.lwaveemit = lwaveemit

    def update_temp(self, surf_resistance, air_temp, dt, lwave):
        def over_write(c,t):
            c.temp = t
        new_temps = calc_new_temp(self.cells, surf_resistance, air_temp, dt, lwave, self.lwaveemit)
        [over_write(c,t) for (c, t) in zip(self.cells, new_temps)]

### 2) Create Envelope Class

#### Function

In [10]:
def make_envelope(nameset, layers, initial_temp, lwaveemit):
    cells = make_cells(layers, initial_temp)
    return Envelope(nameset, cells, lwaveemit)

### 3) Calculate New Temperatures

#### Function

In [11]:
def calc_new_temp(cells, surf_resistance, air_temp, dt, lwave, lwaveemit):

    def calc_longwave_emittance(lwaveemit, surftemp):
        SBFUNC = 5.670367 * 10**(-8)
        TPWATER = 273.15
        return lwaveemit * SBFUNC * (surftemp + TPWATER)**4

    def calc_each_cells(i, cells, surf_resistance, air_temp, dt, lwave, lwaveemit):
        
        if i == 0:
            heat_flow_mns = (air_temp.mns - cells[i].temp) / surf_resistance.mns \
                            + lwave.mns * lwaveemit.mns - calc_longwave_emittance(lwaveemit.mns, cells[i].temp)
        else:
            heat_flow_mns = (cells[i-1].temp - cells[i].temp) / (cells[i-1].resistance.pls + cells[i].resistance.mns)
            
        if i == len(cells) - 1:
            heat_flow_pls = (air_temp.pls - cells[i].temp) / surf_resistance.pls \
                            + lwave.pls * lwaveemit.pls - calc_longwave_emittance(lwaveemit.pls, cells[i].temp)
        else:
            heat_flow_pls = (cells[i+1].temp - cells[i].temp) / (cells[i+1].resistance.mns + cells[i].resistance.pls)

        return (heat_flow_mns + heat_flow_pls) * dt / cells[i].capacity + cells[i].temp

    return [ calc_each_cells(i, cells, surf_resistance, air_temp, dt, lwave, lwaveemit) for i in range(len(cells))]

### 4) Show A Temperature

In [12]:
def show_temperature(envelope, index):
    
    if index >= len(envelope.cells):
        raise Exception('Index is over the number of cells.')
    if index < 0:
        raise Exception('Index should be over zero.')
    
    return envelope.cells[index].temp

### 5) Show Surface Temperatures

In [13]:
def show_surface_temperature(envelope):
    
    return Side(envelope.cells[0].temp, envelope.cells[-1].temp)

### 6) Show All Temperatures

In [14]:
def show_all_temperature(envelope):
    
    return [c.temp for c in envelope.cells]

### 7) Set Surface Temperatures

In [15]:
def set_surface_temperature(envelope, t, sideIndex):
    if sideIndex == 'A':
        envelope.cells[0].temp = t
    elif sideIndex == 'B':
        envelope.cells[-1].temp = t
    else:
        raise ValueError

### 7) Example

In [16]:
e2 = make_envelope('test2',
                   [Solid_Layer(conductivity=1.2, specific_heat=100000.0, thickness=0.12, division_number=3),
                    Air_Layer(resistance=120.0),
                    Solid_Layer(conductivity=1.2, specific_heat=100000.0, thickness=0.12, division_number=3),
                    Solid_Layer(conductivity=1.2, specific_heat=100000.0, thickness=0.12, division_number=3),
                    Air_Layer(resistance=120.0),
                    Solid_Layer(conductivity=1.2, specific_heat=100000.0, thickness=0.12, division_number=6),
                   ],
                   15.0,
                   Side(0.0, 0.0)
                  )

In [17]:
for c in e2.cells:
    print( 'R: ' + str(c.resistance.mns) + '/' + str(c.resistance.pls) + ', capacity: ' + str(c.capacity) + ', temperature: ' + str(c.temp) )

R: surfaceMns/0.03333333333333333, capacity: 2000.0, temperature: 15.0
R: 0.03333333333333333/0.03333333333333333, capacity: 4000.0, temperature: 15.0
R: 0.03333333333333333/0.03333333333333333, capacity: 4000.0, temperature: 15.0
R: 0.03333333333333333/0.03333333333333333, capacity: 4000.0, temperature: 15.0
R: 0.03333333333333333/0.03333333333333333, capacity: 4000.0, temperature: 15.0
R: 0.03333333333333333/0.03333333333333333, capacity: 4000.0, temperature: 15.0
R: 0.03333333333333333/120.0, capacity: 2000.0, temperature: 15.0
R: 120.0/0.03333333333333333, capacity: 2000.0, temperature: 15.0
R: 0.03333333333333333/0.03333333333333333, capacity: 4000.0, temperature: 15.0
R: 0.03333333333333333/0.03333333333333333, capacity: 4000.0, temperature: 15.0
R: 0.03333333333333333/0.03333333333333333, capacity: 4000.0, temperature: 15.0
R: 0.03333333333333333/0.03333333333333333, capacity: 4000.0, temperature: 15.0
R: 0.03333333333333333/0.03333333333333333, capacity: 4000.0, temperature: 15

In [18]:
e2.update_temp(surf_resistance = Side(0.1, 0.1), air_temp = Side(10.0, 10.0), dt = 1.0, lwave = Side(0.0, 0.0))
print(show_all_temperature(e2))
e2.update_temp(surf_resistance = Side(0.1, 0.1), air_temp = Side(10.0, 10.0), dt = 1.0, lwave = Side(0.0, 0.0))
print(show_all_temperature(e2))
e2.update_temp(surf_resistance = Side(0.1, 0.1), air_temp = Side(10.0, 10.0), dt = 1.0, lwave = Side(0.0, 0.0))
print(show_all_temperature(e2))
e2.update_temp(surf_resistance = Side(0.1, 0.1), air_temp = Side(10.0, 10.0), dt = 1.0, lwave = Side(0.0, 0.0))
print(show_all_temperature(e2))
e2.update_temp(surf_resistance = Side(0.1, 0.1), air_temp = Side(10.0, 10.0), dt = 1.0, lwave = Side(0.0, 0.0))
print(show_all_temperature(e2))

[14.975, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 14.95]
[14.950312499999999, 14.99990625, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 14.99925, 14.902]
[14.925932890624999, 14.999720625, 14.9999996484375, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 14.99998875, 14.9978025, 14.8558975]
[14.901856634179687, 14.999444967333984, 14.99999860341797, 14.999999998681641, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 14.99999983125, 14.999956125, 14.99570671875, 14.811595675]
[14.878079263507447, 14.99908108721997, 14.999996532514894, 14.999999993454347, 14.999999999995056, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15